In [ ]:
def basit_finnhub_toplama():
    """Sadece Finnhub'dan haber topla"""
    print("🔧 Finnhub haber toplama başlıyor...")
    
    import requests
    from datetime import datetime, timedelta
    
    api_key = "d3bgdk9r01qqg7bugij0d3bgdk9r01qqg7bugijg"
    
    # Son 3 günün haberlerini çek (sınırlı tut)
    end_date = datetime.now()
    start_date = end_date - timedelta(days=3)
    start_str = start_date.strftime('%Y-%m-%d')
    end_str = end_date.strftime('%Y-%m-%d')
    
    url = f"https://finnhub.io/api/v1/company-news?symbol=AAPL&from={start_str}&to={end_str}&token={api_key}"
    
    try:
        response = requests.get(url, timeout=10)
        
        if response.status_code != 200:
            print(f"❌ Finnhub API hatası: {response.status_code}")
            return []
        
        data = response.json()
        
        haber_listesi = []
        for item in data:
            try:
                # Tarih formatını düzelt
                tarih = datetime.fromtimestamp(item['datetime']).date()
                
                from textblob import TextBlob
                sentiment = TextBlob(item['headline']).sentiment.polarity
                
                haber = {
                    'Tarih': pd.to_datetime(tarih),
                    'Kaynak': 'Finnhub',
                    'Baslik': item['headline'],
                    'Sentiment': sentiment,
                    'URL': item.get('url', '')
                }
                haber_listesi.append(haber)
            except Exception as e:
                continue
        
        print(f"✅ Finnhub: {len(haber_listesi)} haber çekildi")
        return haber_listesi
        
    except Exception as e:
        print(f"❌ Finnhub bağlantı hatası: {e}")
        return []

In [2]:
def basit_yahoo_toplama():
    """Sadece Yahoo RSS'den haber topla"""
    print("🔧 Yahoo RSS haber toplama başlıyor...")
    
    import feedparser
    from textblob import TextBlob
    from datetime import datetime
    
    url = "https://feeds.finance.yahoo.com/rss/2.0/headline?s=AAPL&region=US&lang=en-US"
    
    try:
        feed = feedparser.parse(url)
        
        haber_listesi = []
        for entry in feed.entries[:20]:  # Sadece ilk 20 haber
            try:
                # Tarih formatını düzelt
                tarih = datetime(*entry.published_parsed[:6]).date()
                
                sentiment = TextBlob(entry.title).sentiment.polarity
                
                haber = {
                    'Tarih': pd.to_datetime(tarih),
                    'Kaynak': 'YahooRSS', 
                    'Baslik': entry.title,
                    'Sentiment': sentiment,
                    'URL': entry.link
                }
                haber_listesi.append(haber)
            except Exception as e:
                continue
        
        print(f"✅ Yahoo RSS: {len(haber_listesi)} haber çekildi")
        return haber_listesi
        
    except Exception as e:
        print(f"❌ Yahoo RSS bağlantı hatası: {e}")
        return []

In [3]:
def tum_kaynaklari_toplama():
    """Tüm kaynaklardan haber topla ve tek CSV'ye kaydet"""
    print("🚀 Tüm kaynaklardan haber toplama başlıyor...")
    
    # 1. Tüm kaynaklardan haberleri topla
    finnhub_haberleri = basit_finnhub_toplama()
    yahoo_haberleri = basit_yahoo_toplama()
    
    # 2. Tüm haberleri birleştir
    tum_haberler = []
    tum_haberler.extend(finnhub_haberleri if finnhub_haberleri else [])
    tum_haberler.extend(yahoo_haberleri if yahoo_haberleri else [])
    
    if not tum_haberler:
        print("❌ Hiçbir kaynaktan haber çekilemedi")
        return None
    
    # 3. DataFrame oluştur
    yeni_df = pd.DataFrame(tum_haberler)
    
    # 4. CSV'ye kaydet veya güncelle
    try:
        # Mevcut CSV'yi oku
        existing_df = pd.read_csv('apple_news_archive.csv', parse_dates=['Tarih'])
        
        # Yeni haberleri ekle
        combined_df = pd.concat([existing_df, yeni_df], ignore_index=True)
        
        # Tekrarları temizle (başlığa göre)
        combined_df = combined_df.drop_duplicates(subset=['Baslik'], keep='last')
        
        # Tarihe göre sırala
        combined_df = combined_df.sort_values('Tarih', ascending=False)
        
        # Kaydet
        combined_df.to_csv('apple_news_archive.csv', index=False, encoding='utf-8')
        
        print(f"✅ {len(yeni_df)} yeni haber eklendi. Toplam: {len(combined_df)} haber")
        
    except FileNotFoundError:
        # İlk defa kaydediliyor
        yeni_df.to_csv('apple_news_archive.csv', index=False, encoding='utf-8')
        print(f"✅ İlk kayıt! {len(yeni_df)} haber kaydedildi")
    
    return yeni_df

In [4]:
# ÖNCE TEK TEK TEST ET:
print("\n=== FİNNHUB TEST ===")  
finnhub_df = basit_finnhub_toplama()

print("\n=== YAHOO RSS TEST ===")
yahoo_df = basit_yahoo_toplama()

# HEPSİ ÇALIŞIYORSA:
print("\n=== TÜM KAYNAKLAR TEST ===")
final_df = tum_kaynaklari_toplama()

if final_df is not None:
    print("\n📊 Sonuç:")
    print(f"Toplam haber: {len(final_df)}")
    print("Kaynak dağılımı:")
    print(final_df['Kaynak'].value_counts())


=== FİNNHUB TEST ===
🔧 Finnhub haber toplama başlıyor...
✅ Finnhub: 0 haber çekildi

=== YAHOO RSS TEST ===
🔧 Yahoo RSS haber toplama başlıyor...
✅ Yahoo RSS: 0 haber çekildi

=== TÜM KAYNAKLAR TEST ===
🚀 Tüm kaynaklardan haber toplama başlıyor...
🔧 Finnhub haber toplama başlıyor...
✅ Finnhub: 0 haber çekildi
🔧 Yahoo RSS haber toplama başlıyor...
✅ Yahoo RSS: 0 haber çekildi
❌ Hiçbir kaynaktan haber çekilemedi
